# ATP + Glucose Reaction Thermodynamic Analysis

Using CC method (Component Contribution) to calculate the equilibrium components of the reaction 
'ATP + Glucose = Glucose-6-phosphate + ADP' at 298.15 K, pH 9 and 0.25 M ionic strength.

Initial conditions: Initial concentrations of ATP and glucose are both 0.001 M

In [ ]:
# Import necessary libraries
import subprocess
import json
import sys
import os
from typing import Optional, Tuple, Union

import numpy as np
import numpy.typing as npt
from rdkit import Chem

from equilibrator_api import ComponentContribution, Q_
from scipy.optimize import fsolve
from scipy.constants import R
import math

print("Initializing ComponentContribution instance...")

# Initialize CC class
cc = ComponentContribution()
print("ComponentContribution instance created")

# Set reaction conditions
p_h = 9.0
p_mg = 3.0  # default value
I = 0.25  # ionic strength (M)
T = 298.15  # temperature (K)

cc.p_h = Q_(p_h)
cc.p_mg = Q_(p_mg)
cc.ionic_strength = Q_(f'{I}M')
cc.temperature = Q_(f'{T}K')

print(f'Setting conditions: pH={p_h}, pMg={p_mg}, ionic strength={I}M, temperature={T}K')

# Define reaction, using KEGG IDs
# ATP: C00002, D-glucose: C00031, D-glucose 6-phosphate: C00092, ADP: C00008
reaction_formula = 'C00002 + C00031 = C00092 + C00008'
print(f'Parsing reaction: {reaction_formula}')

# Parse reaction
parsed_rxn = cc.parse_reaction_formula(reaction_formula)
print('Reaction parsing completed')
print('Reaction:', str(parsed_rxn))

# Calculate standard reaction free energy change
dg_prime = cc.standard_dg_prime(parsed_rxn)
print(f'Standard reaction free energy change: {dg_prime}')
print(f'Delta G\'° value (kJ/mol): {dg_prime.value.m:.2f}')

# Calculate equilibrium constant from reaction free energy
T = 298.15  # K
delta_g_prime_kj_per_mol = dg_prime.value.m  # kJ/mol
delta_g_prime_j_per_mol = delta_g_prime_kj_per_mol * 1000  # J/mol

# Calculate equilibrium constant K_eq
RT = R * T
ln_keq = -delta_g_prime_j_per_mol / RT
K_eq = math.exp(ln_keq)

print(f"Equilibrium constant K_eq = {K_eq:.2e}")

# Calculate equilibrium concentrations of each component
# For the reaction ATP + Glucose = Glucose-6-phosphate + ADP
# Let x be the extent of reaction
# [ATP] = 0.001 - x
# [Glucose] = 0.001 - x
# [Glucose-6-phosphate] = x
# [ADP] = x

# Equilibrium equation: K_eq = ([G6P][ADP])/([ATP][glucose]) = x^2 / ((0.001-x)^2)
# Analytical solution: sqrt(K_eq) = x / (0.001-x)
# sqrt(K_eq) * (0.001-x) = x
# sqrt(K_eq) * 0.001 - sqrt(K_eq) * x = x
# sqrt(K_eq) * 0.001 = x + sqrt(K_eq) * x = x * (1 + sqrt(K_eq))
# x = sqrt(K_eq) * 0.001 / (1 + sqrt(K_eq))
sqrt_keq = math.sqrt(K_eq)
x_solution = sqrt_keq * 0.001 / (1 + sqrt_keq)
print(f"Analytical solution: x = {x_solution:.6f} M")
    
# Calculate equilibrium concentrations of each component
atp_eq = 0.001 - x_solution
glucose_eq = 0.001 - x_solution
g6p_eq = x_solution
adp_eq = x_solution

print(f"\nEquilibrium concentrations of each component:")
print(f"[ATP] = {atp_eq:.6f} M")
print(f"[Glucose] = {glucose_eq:.6f} M")
print(f"[Glucose-6-phosphate] = {g6p_eq:.6f} M")
print(f"[ADP] = {adp_eq:.6f} M")

# Verify equilibrium constant
if atp_eq > 0 and glucose_eq > 0:
    calculated_keq = (g6p_eq * adp_eq) / (atp_eq * glucose_eq)
    print(f"\nVerification: Calculated equilibrium constant = {calculated_keq:.2e}")
    print(f"Original equilibrium constant = {K_eq:.2e}")
    print(f"Error: {abs(calculated_keq - K_eq) / K_eq * 100:.4f}%")
else:
    print("Cannot verify, as some reactant concentrations approach 0")

print(f"\nConclusion: Under 298.15 K, pH 9 and 0.25 M ionic strength,")
print(f"for the reaction 'ATP + Glucose = Glucose-6-phosphate + ADP',")
print(f"when the initial concentrations of ATP and glucose are both 0.001 M,")
print(f"the final equilibrium concentrations of components are:")
print(f"ATP: {atp_eq:.6f} M")
print(f"Glucose: {glucose_eq:.6f} M")
print(f"Glucose-6-phosphate: {g6p_eq:.6f} M")
print(f"ADP: {adp_eq:.6f} M")

Initializing ComponentContribution instance...
ComponentContribution instance created
Setting conditions: pH=9.0, pMg=3.0, ionic strength=0.25M, temperature=298.15K
Parsing reaction: C00002 + C00031 = C00092 + C00008
Reaction parsing completed
Reaction: Compound(id=6, inchi_key=ZKHQWZAMYRWXGA-KQYNXXCUSA-J) + Compound(id=43, inchi_key=WQZGKKKJIJFFOK-GASJEMHNSA-N) = Compound(id=10, inchi_key=XTWYTFMLZFPYCI-KQYNXXCUSA-K) + Compound(id=152, inchi_key=NBSCHQHZLSJFNQ-GASJEMHNSA-L)
Standard reaction free energy change: (-28.8 +/- 0.4) kilojoule / mole
Delta G'° value (kJ/mol): -28.85
Equilibrium constant K_eq = 1.13e+05
Analytical solution: x = 0.000997 M

Equilibrium concentrations of each component:
[ATP] = 0.000003 M
[Glucose] = 0.000003 M
[Glucose-6-phosphate] = 0.000997 M
[ADP] = 0.000997 M

Verification: Calculated equilibrium constant = 1.13e+05
Original equilibrium constant = 1.13e+05
Error: 0.0000%

Conclusion: Under 298.15 K, pH 9 and 0.25 M ionic strength,
for the reaction 'ATP + G

# Analysis of pMg's effect on ATP equilibrium concentration

In [ ]:
# Import plotting library and set font
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import font_manager

# For Chinese font issue, we'll use English labels

# Read pMg analysis data
data_file = 'atp_concentration_vs_pMg.csv'
df = pd.read_csv(data_file, encoding='utf-8')

# Confirm column names
print('Column names in the data file:', df.columns.tolist())

# Plot graph of ATP concentration vs pMg
plt.figure(figsize=(12, 8))
plt.plot(df['pMg'], df['ATP平衡浓度 (M)'], 'b-o', markersize=4, linewidth=1.5)
plt.xlabel('pMg')
plt.ylabel('ATP equilibrium concentration (M)')
plt.title('ATP equilibrium concentration at different pMg (298.15 K, pH 9, 0.25 M ionic strength)')
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Use logarithmic scale to better show concentration changes

# Mark key points
min_idx = df['ATP平衡浓度 (M)'].idxmin()
max_idx = df['ATP平衡浓度 (M)'].idxmax()

plt.annotate(f'Min: ({df.loc[min_idx, "pMg"]:.1f}, {df.loc[min_idx, "ATP平衡浓度 (M)"]:.2e})',
             xy=(df.loc[min_idx, 'pMg'], df.loc[min_idx, 'ATP平衡浓度 (M)']),
             xytext=(df.loc[min_idx, 'pMg']+1.5, df.loc[min_idx, 'ATP平衡浓度 (M)']*2),
             arrowprops=dict(arrowstyle='->', color='red', lw=0.8))

plt.annotate(f'Max: ({df.loc[max_idx, "pMg"]:.1f}, {df.loc[max_idx, "ATP平衡浓度 (M)"]:.2e})',
             xy=(df.loc[max_idx, 'pMg'], df.loc[max_idx, 'ATP平衡浓度 (M)']),
             xytext=(df.loc[max_idx, 'pMg']-2, df.loc[max_idx, 'ATP平衡浓度 (M)']/2),
             arrowprops=dict(arrowstyle='->', color='red', lw=0.8))

# Draw physiological condition annotation line
physiological_pmg_idx = (df['pMg'] - 3.0).abs().idxmin()
plt.axvline(x=3.0, color='green', linestyle='--', alpha=0.6, label=f'physiological pMg={df.loc[physiological_pmg_idx, "pMg"]:.1f}')
plt.legend()

plt.tight_layout()
plt.savefig('atp_concentration_vs_pMg_plot_notebook.png', dpi=300, bbox_inches='tight', 
            facecolor='white', edgecolor='none')
print('\nGraph saved as atp_concentration_vs_pMg_plot_notebook.png')

plt.show()

# Display first few rows of the data table
print('pMg effect on ATP equilibrium concentration data:')
print(df.head(10))

# Output key information
print(f'\nKey information:')
print(f'ATP concentration at physiological conditions (pMg=3): {df.loc[physiological_pmg_idx, "ATP平衡浓度 (M)"]:.8f} M')
print(f'Minimum ATP concentration: {df["ATP平衡浓度 (M)"].min():.8f} M (at pMg={df.loc[df["ATP平衡浓度 (M)"].idxmin(), "pMg"]})')
print(f'Maximum ATP concentration: {df["ATP平衡浓度 (M)"].max():.8f} M (at pMg={df.loc[df["ATP平衡浓度 (M)"].idxmax(), "pMg"]})')

# Analysis of combined effects of pH and pMg on ATP equilibrium concentration

In [ ]:
# Read 2D analysis data for pH and pMg
df_2d = pd.read_csv('atp_concentration_vs_ph_pmg.csv', encoding='utf-8')

# Display first few rows of data
print('Data for pH and pMg effects on ATP equilibrium (first 10 rows):')
print(df_2d.head(10))

# Extract unique pH and pMg values from the data
pH_values = sorted(df_2d['pH'].unique())
pMg_values = sorted(df_2d['pMg'].unique())

# Reshape 2D arrays
Z_atp = df_2d['ATP平衡浓度 (M)'].values.reshape(len(pH_values), len(pMg_values))
Z_dg = df_2d['ΔG\'° (kJ/mol)'].values.reshape(len(pH_values), len(pMg_values))

print(f'\nData dimensions: pH {len(pH_values)} values × pMg {len(pMg_values)} values')
print(f'ATP equilibrium concentration range: {df_2d["ATP平衡浓度 (M)"].min():.2e} - {df_2d["ATP平衡浓度 (M)"].max():.2e} M')
print(f'ΔG\'° range: {df_2d["ΔG\'° (kJ/mol)"].min():.2f} - {df_2d["ΔG\'° (kJ/mol)"].max():.2f} kJ/mol')

# Create heatmap for ATP equilibrium concentration
fig, ax = plt.subplots(figsize=(12, 8))

# Create heatmap
im = ax.imshow(Z_atp, cmap='viridis', aspect='auto', 
               extent=[pMg_values[0], pMg_values[-1], pH_values[0], pH_values[-1]], 
               origin='lower', vmin=df_2d['ATP平衡浓度 (M)'].min(), vmax=df_2d['ATP平衡浓度 (M)'].max())

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('ATP equilibrium concentration (M)', fontsize=12)

# Set axes
ax.set_xlabel('pMg', fontsize=12)
ax.set_ylabel('pH', fontsize=12)
ax.set_title('Heatmap of ATP equilibrium concentration vs pH and pMg (298.15 K, 0.25 M ionic strength)', fontsize=14)

plt.tight_layout()
plt.savefig('atp_concentration_heatmap_notebook.png', dpi=300, bbox_inches='tight')
print('\nATP equilibrium concentration heatmap saved as atp_concentration_heatmap_notebook.png')
plt.show()

# Create heatmap for ΔG'°
fig, ax = plt.subplots(figsize=(12, 8))

# Create heatmap
im2 = ax.imshow(Z_dg, cmap='RdBu_r', aspect='auto', 
                extent=[pMg_values[0], pMg_values[-1], pH_values[0], pH_values[-1]], 
                origin='lower', vmin=df_2d['ΔG\'° (kJ/mol)'].min(), vmax=df_2d['ΔG\'° (kJ/mol)'].max())

# Add colorbar
cbar2 = plt.colorbar(im2, ax=ax)
cbar2.set_label('ΔG\'° (kJ/mol)', fontsize=12)

# Set axes
ax.set_xlabel('pMg', fontsize=12)
ax.set_ylabel('pH', fontsize=12)
ax.set_title('Heatmap of standard reaction free energy change vs pH and pMg (298.15 K, 0.25 M ionic strength)', fontsize=14)

plt.tight_layout()
plt.savefig('delta_g_heatmap_notebook.png', dpi=300, bbox_inches='tight')
print('ΔG\'° heatmap saved as delta_g_heatmap_notebook.png')
plt.show()

# Find values under physiological conditions (pH≈7.4, pMg≈3.0)
physio_row = df_2d.iloc[(abs(df_2d['pH'] - 7.4) + abs(df_2d['pMg'] - 3.0)).idxmin()]
print(f'\nPhysiological condition estimate (pH≈{physio_row["pH"]:.1f}, pMg≈{physio_row["pMg"]:.1f}):')
print(f'ATP equilibrium concentration: {physio_row["ATP平衡浓度 (M)"]:.2e} M')
print(f'ΔG\'°: {physio_row["ΔG\'° (kJ/mol)"]:.2f} kJ/mol')

# Summary and Biological Significance

## Key Findings

1. Under conditions of pH 9, pMg 3.0, and ionic strength 0.25 M:
   - ΔG'° = -28.85 kJ/mol
   - Equilibrium constant K_eq = 1.13×10^5
   - ATP equilibrium concentration: 0.000003 M (almost complete reaction)

2. Effect of pMg on ATP equilibrium concentration:
   - ATP equilibrium concentration decreases with increasing pMg
   - ATP concentration at physiological pMg(3.0): 0.00000296 M

3. 2D parameter effects (pH-pMg):
   - Analysis of 99 data points (pH 5.0-10.0, pMg 2.0-10.0)
   - ATP equilibrium concentration range: 4.56×10⁻⁷ M - 8.88×10⁻⁵ M
   - Physiological condition estimate (pH≈7.5, pMg≈3.0): [ATP]≈1.61×10⁻⁵ M

## Biological Significance

1. The reaction has strong thermodynamic driving force under physiological conditions
2. Mg²⁺ ions significantly affect ATP stability
3. Cells can regulate the equilibrium position of metabolic reactions by adjusting pH and Mg²⁺ concentration
4. This dual regulatory mechanism provides cells with fine metabolic control strategies